In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot

In [3]:
new_model = tf.keras.models.load_model("D:/Modelos/ResNet50_IC_copy")

In [4]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 1, 1, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 2

In [5]:
#carrega os arrays de imagens e de classes
train_X = np.load("D:/Displasia/displasiaDataAug/displasiaOriginalDataX_teste3.npy")
train_label = np.load("D:/Displasia/displasiaDataAug/displasiaOriginalDataYOneHot_teste3.npy")
XSCM = np.load("D:/Displasia/displasiaDataAug/displasiaOriginalDataX_teste3_SCM.npy")
#XGLCM = np.load("D:/Displasia/displasiaDataAug/displasiaOriginalDataX_teste3_GLCM.npy")
#dataX = np.load("Image Arrays/displasiaGeneratedDataX.npy")
#dataY = np.load("Image Arrays/displasiaGeneratedDataY.npy")

#XSCM = np.append(XSCM,XGLCM,axis=1)

print(XSCM.shape)

#shape dos arrays
#train_X.shape, train_label.shape
#shape dos arrays
train_X.shape, train_label.shape

(75776, 8)


((75776, 32, 32, 3), (75776, 4))

In [6]:
#from sklearn.model_selection import train_test_split

#divide os dados em 80% para treino e 20% para teste
#train_X, test_X, train_label, test_label = train_test_split(X, Y, test_size=0.2, random_state=13)

#divide os dados de treino em 80% para treino e 20% para validação
#train_X, valid_X, train_label, valid_label = train_test_split(train_X, train_label, test_size=0.2, random_state=13)

#mostra os shapes resultantes das divisões
#print("Training:   ", train_X.shape, train_label.shape)
#print("Validation: ", valid_X.shape, valid_label.shape)
#print("Testing:    ", test_X.shape, test_label.shape)

In [7]:
#divide os dados em 80% para treino e 20% para teste
#train_X_SCM, test_X_SCM, train_label_SCM, test_label_SCM = train_test_split(XSCM, Y, test_size=0.2, random_state=13)

#divide os dados de treino em 80% para treino e 20% para validação
#train_X_SCM, valid_X_SCM, train_label_SCM, valid_label_SCM = train_test_split(train_X_SCM, train_label_SCM, test_size=0.2, random_state=13)

#mostra os shapes resultantes das divisões
#print("Training:   ", train_X_SCM.shape, train_label_SCM.shape)
#print("Validation: ", valid_X.shape, valid_label.shape)
#print("Testing:    ", test_X_SCM.shape, test_label_SCM.shape)

In [8]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import keras
from sklearn.model_selection import KFold
#kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
kfold = KFold(n_splits=5, shuffle=True, random_state=7)
cvscores = []

mean = [[[],[],[],[],[]],
         [[],[],[],[],[]],
         [[],[],[],[],[]],
         [[],[],[],[],[]]]

std = [[[],[],[],[],[]],
         [[],[],[],[],[]],
         [[],[],[],[],[]],
         [[],[],[],[],[]]]

for train, test in kfold.split (train_X, train_label):

    intermediate_layer_model = keras.Model(inputs=new_model.input,
                                           outputs=new_model.get_layer('flatten').output)
    intermediate_output = intermediate_layer_model.predict(train_X[train])

    newArr = np.append(intermediate_output, XSCM[train] , axis=1)

    clf=RandomForestClassifier(n_estimators=100)
    #train_labels2=train_labels[:49984]
    clf.fit(newArr,train_label[train])

    intermediate_layer_model_2 = keras.Model(inputs=new_model.input,
                                           outputs=new_model.get_layer('flatten').output)
    intermediate_output_2 = intermediate_layer_model_2.predict(train_X[test])

    newArr_test = np.append(intermediate_output_2, XSCM[test] , axis=1)

    y_pred=clf.predict(newArr_test)

    # Model Accuracy, how often is the classifier correct?
    print("Accuracy:",metrics.accuracy_score(train_label[test], y_pred))
    cvscores.append(metrics.accuracy_score(train_label[test], y_pred)*100)
    
    from keras.utils import to_categorical

    predicted_classes = y_pred

    predicted_classes = np.argmax(np.round(predicted_classes),axis=1)

    predicted_classes = to_categorical(predicted_classes)

    
    num_classes = 4
    from sklearn.metrics import classification_report
    target_names = ["Class {}".format(i) for i in range(num_classes)]
    print(classification_report(train_label[test], predicted_classes, target_names=target_names))
    
    from sklearn.metrics import confusion_matrix

    #converte os arrays de classe de one-hot para númerico
    test_label_aux = np.argmax(train_label[test], axis=1)
    predicted_classes = np.argmax(predicted_classes, axis=1)

    #gera a matriz de confusão a partir da predição feita
    cm = confusion_matrix(test_label_aux, predicted_classes)

    #gera os valores de falso positivo, falso negativo, verdadeiro positivo e verdadeiro negativo
    fp = cm.sum(axis=0) - np.diag(cm)  
    fn = cm.sum(axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tn = cm.sum() - (fp + fn + tp)

    #gera a sensitividade, especificidade, acurácia e precisão
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy    = (tp + tn) / (tp + tn + fp + fn)
    precision   = tp / (tp + fp)
    f1_score = 2/((1/precision)+(1/sensitivity))

    #mostra os resultados
    print("Precision:  ", precision)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)
    print("F1-score:   ", f1_score)
    print("Accuracy:   ", accuracy)
    
    x = []
    x.append(precision)
    x.append(sensitivity)
    x.append(specificity)
    x.append(f1_score)
    x.append(accuracy)
    
    x = np.asarray(x)
    
    x = x.transpose()
    print()
    print(x)
    
    for i in range(4):
        for j in range(5):
            mean[i][j].append(x[i][j])
            std[i][j].append(x[i][j])
    
print("%.2f%% (+/- %.2f%%)" %(np.mean(cvscores),np.std(cvscores)))


Accuracy: 0.9455001319609395
              precision    recall  f1-score   support

     Class 0       0.92      0.99      0.95      3867
     Class 1       0.92      0.93      0.93      3755
     Class 2       0.96      0.93      0.94      3686
     Class 3       0.98      0.94      0.96      3848

   micro avg       0.95      0.95      0.95     15156
   macro avg       0.95      0.95      0.95     15156
weighted avg       0.95      0.95      0.95     15156
 samples avg       0.95      0.95      0.95     15156

Precision:   [0.9243453  0.92495359 0.96040438 0.98189189]
Sensitivity: [0.98577709 0.92889481 0.92783505 0.94412682]
Specificity: [0.97236248 0.97517762 0.98770706 0.99407499]
F1-score:    [0.95407333 0.92692001 0.94383883 0.96263911]
Accuracy:    [0.97578517 0.96371074 0.97314595 0.98139351]

[[0.9243453  0.98577709 0.97236248 0.95407333 0.97578517]
 [0.92495359 0.92889481 0.97517762 0.92692001 0.96371074]
 [0.96040438 0.92783505 0.98770706 0.94383883 0.97314595]
 [0.98189189

In [9]:
mean = np.asarray(mean)
std = np.asarray(std)
for i in mean:
    print(np.mean(i,axis=1))
print()
for i in std:
    print(np.std(i,axis=1))

[0.92204617 0.98293735 0.97230144 0.95151699 0.97496568]
[0.9257549  0.92598828 0.97524144 0.92586271 0.96293021]
[0.95911838 0.92652497 0.98683816 0.94253887 0.97175885]
[0.97699575 0.94547606 0.99259795 0.96097063 0.98081186]

[0.00251777 0.00164453 0.00099605 0.00178406 0.00086428]
[0.00216052 0.00593008 0.00087537 0.00342488 0.00166137]
[0.00335181 0.00209385 0.00107449 0.00256731 0.00132993]
[0.00449649 0.00106391 0.00134465 0.0018996  0.00071746]
